# Author: Jason Pranata

In [ ]:
from ultralytics import YOLO
import os
import cv2
import torch

In [ ]:
train_images_path = 'train_dataset/Images'
val_images_path = 'val_dataset/Images'
test_images_path = 'test_dataset/Images'
output_train_images_path = 'train_dataset/resized_Images'
output_val_images_path = 'val_dataset/resized_Images'
output_test_images_path = 'test_dataset/resized_Images'

### Adjust dataset images

In [ ]:
def resize_images(input_path, output_path, target_size=(1024, 1024)):
    for image_file in os.listdir(input_path):
        if not image_file.endswith('.jpg') and not image_file.endswith('.jpeg') and not image_file.endswith('.JPG'):
            continue
        img = cv2.imread(os.path.join(input_path, image_file))
        img_resized = cv2.resize(img, target_size)
        cv2.imwrite(os.path.join(output_path, image_file), img_resized)

In [ ]:
#resize_images(train_images_path, output_train_images_path)
#resize_images(val_images_path, output_val_images_path)
#resize_images(test_images_path, output_test_images_path)

### Visualization

In [ ]:
def draw_bounding_boxes(image_path, annotation_path, output_path):
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    with open(annotation_path, 'r') as file:
        annotations = file.readlines()

    for annotation in annotations:
        class_id, x_center, y_center, bbox_width, bbox_height = map(float, annotation.strip().split())

        x_center *= width
        y_center *= height
        bbox_width *= width
        bbox_height *= height

        x1 = int(x_center - bbox_width / 2)
        y1 = int(y_center - bbox_height / 2)
        x2 = int(x_center + bbox_width / 2)
        y2 = int(y_center + bbox_height / 2)

        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, str(int(class_id)), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    cv2.imwrite(output_path, image)

def check_labels():
    for i in range(0, 1):
        num = i
        image_path = f'val_dataset/resized_Images/AG_0{num}.jpg'
        annotation_path = f'val_dataset/labels/AG_0{num}.txt'
        output_path = f'valD_image{num}.jpg'
        draw_bounding_boxes(image_path, annotation_path, output_path)

#check_labels()

### Training (manually adjust weights, configs, etc. accordingly)

In [ ]:
model = YOLO('runs/detect/train8/weights/best.pt')

In [ ]:
model.predict('test_dataset/resized_Images/AG_043.jpg', save = True)
model.predict('test_dataset/resized_Images/AD_043.jpg', save = True)

In [ ]:
yaml_path = os.path.abspath('yolov8_transferlearn_config.yaml')
train_results = model.train(data=yaml_path, epochs=300, batch=2, imgsz=1024, device='cuda')

In [ ]:
val_results = model.val(data=yaml_path)

In [ ]:
test_ad = 'test_dataset/resized_Images/AD_043.jpg'
test_ag = 'test_dataset/resized_Images/AG_043.jpg'
model.predict(test_ag, save = True)
model.predict(test_ad, save = True)
model.predict("/mnt/c/Users/jason/GitHubRepos/LectorAI-TextExtraction/tempimages_api/beispiel_form_covered.jpg", save = True)
